In [5]:
import numpy as np
import pandas as pd

from sklearn.metrics import ndcg_score, dcg_score

### ОФФЛАЙН-МЕТРИКИ

#### **NDCG (Нормалтзованный дисконтированный куммултивный выишгрыш)**

позволяет использовать информацию о том, что разные продукты имеют разную степень релевантности, то есть учитывает их рейтинг.

In [2]:
true = np.asarray([[3, 2, 1, 0, 0]])
relevance = np.asarray([[3, 2, 0, 0, 1]])

print(ndcg_score(true, relevance))

0.980840401274087


**Задание 4.5**

Пусть у нас есть реальные оценки, выставленные пользователем, и предсказанные оценки:

Реальные оценки: [2, 4, 1, 1, 1]
Предсказанные оценки: [2, 5, 2, 3, 1]
Вычислите коэффициент NDCG. Округлите результат до двух знаков после точки-разделителя.

In [4]:
true = np.asarray([[2, 4, 1, 1, 1]])
relevance = np.asarray([[2, 5, 2, 3, 1]])

print(ndcg_score(true, relevance).round(2))

0.97


### ОНЛАЙН-МЕТРИКИ

### **Бизнес-показатели**


- Продажи и выручка
- Поведение пользователей
- Охват различных категорий товаров
- Применение и конверсия
- СTR (кликабельность)


## Холодный старт. Popularity-based model

Виды проблем холодного старта:

- холодный старт пользователя;
- холодный старт продукта.

In [22]:
url_m = 'https://lms-cdn.skillfactory.ru/assets/courseware/v1/4dc5c1a1e1cc649c1c263450521a00b1/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/movie.zip'
ratings = pd.read_csv('data/dst-3.0_mathml_14_5_rating.csv')
movies = pd.read_csv(url_m)

In [23]:
ratings

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [24]:
df = pd.merge(ratings,movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


**Задание 5.1**

Подсчитайте, сколько раз каждый фильм встречается в наборе данных. Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.

In [26]:
df.title.value_counts()[:5]

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: count, dtype: int64

**Задание 5.2**

Найдите средний рейтинг для каждого из фильмов.
Найдите фильмы с наивысшим средним рейтингом.
Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.
Впишите только название фильма, без кавычек и без года выхода на экран.

In [43]:
df_top = df.groupby('title').rating.mean().reset_index()
df_top[df_top.rating == 5].title.sort_values(ascending=False).head(1).values

array(['Yonkers Joe (2008)'], dtype=object)

**Задание 5.3**

В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

In [63]:
fifty_pop = df.groupby('title').rating.count()[df.groupby('title').rating.count() > 50].index#.reset_index()
len(fifty_pop)

10472

**Задание 5.4**

Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, и найдите среди них фильм с наивысшей средней оценкой. В качестве ответа запишите название этого фильма без артикля и года выхода на экран.

In [70]:
df[df.title.isin(fifty_pop)].groupby('title').rating.mean().sort_values(ascending=False).index[:1]


Index(['Shawshank Redemption, The (1994)'], dtype='object', name='title')